In [2]:
import pandas as pd
 
df = pd.read_csv('/Users/mustakahmad/Library/CloudStorage/OneDrive-purdue.edu/FACAI LAB/Project3_tariff/tariff_war/raw_data/soybean/Brazil_Forest_Loss_2013_2022_1km.csv')

In [20]:
df1 = pd.read_csv('/Users/mustakahmad/Downloads/ForestExtent2020_Points_3km.csv')
df1

,ID,LON,LAT,FH2020
0,1,123.795,72.065,6.536736
1,2,123.825,72.065,6.030208
2,3,123.855,72.065,7.494861
3,4,99.225,71.945,5.734792
4,5,101.805,71.915,5.011111
...,...,...,...,...
5122198,5122199,-67.815,-55.855,5.212430
5122199,5122200,-67.725,-55.855,5.632153
5122200,5122201,-67.695,-55.855,5.775208
5122201,5122202,-67.605,-55.855,9.615278


In [11]:
import ee
ee.Initialize(project= 'forest-cover-461900')

# 1. Define Brazil boundary
brazil = ee.FeatureCollection("FAO/GAUL/2015/level0") \
           .filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))

# 2. Load Hansen forest change dataset
hansen = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')

# 3. Select relevant bands
treecover2000 = hansen.select('treecover2000')
loss = hansen.select('loss')
lossyear = hansen.select('lossyear')
gain = hansen.select('gain')

# 4. Mask forest loss from 2015 to 2022
loss_mask = lossyear.gte(15).And(lossyear.lte(22))
loss_filtered = lossyear.updateMask(loss_mask)

# 5. Convert loss year to actual year (2000 + value)
loss_year_actual = loss_filtered.add(2000).rename('loss_year')

# 6. Stack all required bands
stack = treecover2000.addBands(loss).addBands(gain).addBands(loss_year_actual)

# 7. Clip to Brazil and reproject
stack_clipped = stack.clip(brazil).reproject(crs='EPSG:4326', scale=1000)

# 8. Sample pixels
samples = stack_clipped.sample(
    region=brazil.geometry(),
    scale=1000,
    geometries=True
)

# 9. Export to Google Drive as CSV
task = ee.batch.Export.table.toDrive(
    collection=samples,
    description='Brazil_Forest_Loss_2015_2022',
    fileFormat='CSV'
)

task.start()
print("Export started. Check the GEE Code Editor 'Tasks' tab or Google Drive.")

Export started. Check the GEE Code Editor 'Tasks' tab or Google Drive.


In [25]:
import ee
ee.Initialize(project='forest-cover-461900')  # Ensure your GEE project is set

# Brazil boundary
brazil = ee.FeatureCollection("FAO/GAUL/2015/level0") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))

# Grid points
grid_points = ee.FeatureCollection('projects/forest-cover-461900/assets/ForestExtent2020_Points_3km')
points_in_brazil = grid_points.filterBounds(brazil)

# Hansen dataset
hansen = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')

# Reproject Hansen bands to 3km scale with SRTM projection
srtm = ee.Image('CGIAR/SRTM90_V4')
projection = srtm.reproject('EPSG:4326', None, 3000).projection()

# Reproject bands
tree_cover = hansen.select('treecover2000').reproject(projection)
loss = hansen.select('loss').reproject(projection)
gain = hansen.select('gain').reproject(projection)
loss_year = hansen.select('lossyear').reproject(projection)

def add_forest_attributes(feature):
    geom = feature.geometry()

    cover = tree_cover.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=geom,
        scale=3000
    ).get('treecover2000')

    loss_val = loss.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=geom,
        scale=3000
    ).get('loss')

    gain_val = gain.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=geom,
        scale=3000
    ).get('gain')

    loss_year_val = loss_year.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=geom,
        scale=3000
    ).get('lossyear')

    loss_year_val = ee.Number(loss_year_val)
    actual_loss_year = loss_year_val.gt(0).multiply(loss_year_val.add(2000)).int()

    # Loss per hectare: 1 pixel is 30m x 30m = 900 m² = 0.09 ha
    # But here at 3000m scale, pixel area is 3000*3000=9,000,000 m² = 900 ha
    # So loss value (0 or 1) divided by pixel area in ha
    loss_per_ha = ee.Number(loss_val).divide(900)

    return feature.set({
        'treecover2000': cover,
        'loss': loss_val,
        'gain': gain_val,
        'loss_year': actual_loss_year,
        'loss_per_ha': loss_per_ha
    })

points_with_attributes = points_in_brazil.map(add_forest_attributes)

# Export
task = ee.batch.Export.table.toDrive(
    collection=points_with_attributes,
    description='Brazil_ForestAttributes_3kmGrid_Reprojected',
    fileFormat='CSV',
    selectors=['ID', 'treecover2000', 'loss', 'gain', 'loss_year', 'loss_per_ha']
)
task.start()

print('Export started. Monitor the task in the Earth Engine Tasks tab or via task.status()')

Export started. Monitor the task in the Earth Engine Tasks tab or via task.status()


In [3]:
data = pd.read_csv('/Users/mustakahmad/Library/CloudStorage/OneDrive-purdue.edu/FACAI LAB/Project3_tariff/tariff_war/raw_data/Amazon_Forest_Loss_Per_Year_CSV.csv')

In [6]:
data.describe()

,system:index,lat,lon,loss_2013,loss_2014,loss_2015,loss_2016,loss_2017,loss_2018,loss_2019,loss_2020,loss_2021,loss_2022
count,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06,3.223165e+06
mean,1.611582e+06,-6.332462e+00,-6.154989e+01,4.109129e-02,4.950631e-02,2.918622e-02,7.065384e-02,7.329411e-02,3.930298e-02,4.490338e-02,4.903534e-02,4.309646e-02,5.438629e-02
std,9.304477e+05,6.434909e+00,8.569812e+00,1.985014e-01,2.169227e-01,1.683282e-01,2.562458e-01,2.606187e-01,1.943149e-01,2.070920e-01,2.159419e-01,2.030743e-01,2.267784e-01
min,0.000000e+00,-2.049004e+01,-7.969861e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.057910e+05,-1.133621e+01,-6.753542e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.611582e+06,-6.745815e+00,-6.119331e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.417373e+06,-1.814065e+00,-5.456375e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.223164e+06,8.651309e+00,-4.450261e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
